In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
path_to_file = 'shakespeare.txt'
text = open(path_to_file,'r').read()

In [3]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [4]:
vocab = sorted(set(text))
print(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


In [5]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [6]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [7]:
line = 'From fairest creatures we desire increase'
lines = '''
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  '''
print(len(line),len(lines))

41 135


In [8]:
seq_len=120
total_num_seq = len(text)//(seq_len+1)
print(total_num_seq)

8665


In [9]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
 #drop last remaining strings
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [10]:
def create_seq_targets(seq):
  input_txt = seq[:-1] #hello my nam
  target_txt = seq[1:] #ello my name
  return input_txt, target_txt

In [11]:
dataset = sequences.map(create_seq_targets)
batch_size=128
buffer_size = 10000 #memory allocation
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

The input is a sequence of encoded numbers that is related to an original string and the target is that sequence shifted by a number (Again related to a string)

In [12]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

In [13]:
vocab_size = len(vocab)
#embed_dim: integers turned to vectors , relative same with the vocabulary size
embed_dim = 64
rnn_neurons = 1026
#labels are one-hot encoded
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [15]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [16]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [17]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 82)  <=== (batch_size, sequence_length, vocab_size)


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [20]:
#reshape
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
epochs = 30
model.fit(dataset,epochs=epochs)

Epoch 1/30
67/67 [==============================] - 10s 115ms/step - loss: 3.6038
Epoch 2/30
67/67 [==============================] - 9s 116ms/step - loss: 2.7229
Epoch 3/30
67/67 [==============================] - 9s 118ms/step - loss: 2.3193
Epoch 4/30
67/67 [==============================] - 9s 119ms/step - loss: 2.1426
Epoch 5/30
67/67 [==============================] - 9s 122ms/step - loss: 2.0137
Epoch 6/30
67/67 [==============================] - 9s 122ms/step - loss: 1.8908
Epoch 7/30
67/67 [==============================] - 9s 122ms/step - loss: 1.7821
Epoch 8/30
67/67 [==============================] - 9s 123ms/step - loss: 1.6893
Epoch 9/30
67/67 [==============================] - 9s 124ms/step - loss: 1.6104
Epoch 10/30
17/67 [======>.......................] - ETA: 6s - loss: 1.5657

KeyboardInterrupt: ignored

In [22]:
model.save('shakespeare_gen.h5') 

In [23]:
from tensorflow.keras.models import load_model

In [24]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('shakespeare_gen.h5')
model.build(tf.TensorShape([1, None]))

generate text

In [26]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. 
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted character for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [27]:
print(generate_text(model,"flower",gen_size=10000))

flowers,
  Will let my given his die dote; mad no think
    Ramane in leess, the placus in still brink of'll.
ANTIPHOLUS OF TYRARS, I am gome is a farltine une eck
  Tear twentle senves. Bread'd great flam to decure
    Amatine enstrambar fiind how?
  IE.                  Exeunt Caesar andryse
    Go sad: it wound was muphis not, his sine a pads.
  JAQEUS. ag I de wnotide no lifed
    Of wanservill enult.
  TELOPETRS. The promes younge, note I thy pauther oft
    wint made with to sen           Hee to by yet, I shall I did to'gl you'g-
            Seep that be no hove and ear; th'
  I prate then strovy, woran that fathbar'
    That but senfuce; but him chear intery fabert-dent heart a stratient
    Than toolly forsution of mecate of mich habsing lars-intund.
    I'll the wall soifect.      Extur a latger
    Of rach woods a sharker.    That it nom the fasts
    Arafther a Cuearan. But bodshat
    And the flowers an what wencers that I wo let
    Buently be to- poor know' the vawter.
  